In [8]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.6 MB/s eta 0:00:00


In [6]:
!pip install nltk

In [13]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

import matplotlib.pyplot as plt
from wordcloud import WordCloud

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('english')

[nltk_data] Error loading english: Package 'english' not found in
[nltk_data]     index


False

In [2]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/law/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/law/test.csv')

# Data Preprocessing

In [3]:
vectorizer = TfidfVectorizer()
def get_vector(vectorizer, df, train_mode):
    if train_mode:
        X_facts = vectorizer.fit_transform(df['facts'])
    else:
        X_facts = vectorizer.transform(df['facts'])
    X_party1 = vectorizer.transform(df['first_party'])
    X_party2 = vectorizer.transform(df['second_party'])

    X = np.concatenate([X_party1.todense(), X_party2.todense(), X_facts.todense()], axis=1)
    return X

In [4]:
X_train = get_vector(vectorizer, train, True)
Y_train = train["first_party_winner"]
X_test = get_vector(vectorizer, test, False)

In [5]:
X_train.shape, Y_train.shape, X_test.shape

((2478, 52377), (2478,), (1240, 52377))

# Define Model & Train (LSTM)

In [6]:
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, Dropout, Activation, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MSE

In [7]:
# LSTM
model = Sequential()

model.add(LSTM(units=128, input_shape=(52377, 1)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               66560     
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 74881 (292.50 KB)
Trainable params: 74881 (292.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(patience=5)

In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=3)#, callbacks=['early_stop'])

Epoch 1/3
78/78 [==============================] - 111s 1s/step - loss: 0.6391 - accuracy: 0.6655
Epoch 2/3
78/78 [==============================] - 107s 1s/step - loss: 0.6393 - accuracy: 0.6655
Epoch 3/3
78/78 [==============================] - 107s 1s/step - loss: 0.6384 - accuracy: 0.6655


# Inference & Submisson

In [13]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/law/sample_submission.csv')

In [14]:
pred = model.predict(X_test)

39/39 [==============================] - 26s 601ms/step


In [15]:
submit['first_party_winner'] = pred
submit.to_csv('./baseline_submit.csv', index=False)
print('Done')

Done
